# Notebook Summary

The purpose of this notebook is to implement a trading strategy where you're buying/selling dogecoin based on the day of the week. 

Analysis of historical dogecoin data with fbprophet indicates that a viable strategy could be to purchase dogecoin on Tuesday and sell it on Thursday. 

# Save Alpaca Credentials in Your .env File

Make sure you have a .env file set up with your Alpaca API keys. The notebook is looking for your Alpaca API keys in this format: 

API_KEY = "xxx"

API_SECRET = "xxx"

# Paper Trading Account Demos

This notebook connects to your PAPER trading account with this url: 

https://paper-api.alpaca.markets

If you wish to test these functions with your LIVE account, change the url and update your API keys if needed. 

(These functions are taken mainly from the Alpaca Documentation with links below)

In [191]:
import pandas as pd
from twarc import Twarc2, expansions
import json
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame, TimeFrameUnit
import os
import datetime
import tweepy
import requests
from datetime import datetime, timedelta


In [188]:
# Load the environment variables from the .env file
load_dotenv()


True

# Create a Connection to the (Paper) Trading API

In [150]:
API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets" #notice "PAPER" in the URL

In [151]:
# Create a connection to the API 
api = tradeapi.REST(API_KEY, API_SECRET, ALPACA_API_BASE_URL, api_version="v2")

# Determine Action (Buy / Sell / NA)

Buy Dogecoin on Tuesday, Sell Dogecoin on Thursday, Make no action in any other day of the week

Assumptions: 

The notebook assumes that you have dogecoin in your account to sell

The notebook is using increments of 1000 Dogecoins (which as of 9/22/2022 is $60 worth)

In [201]:
#https://stackoverflow.com/questions/9847213/how-do-i-get-the-day-of-week-given-a-date
#monday is 0, sunday is 6
#buy on Tuesday (1) and sell on Thursday (3)

action = "none"

day_of_the_week = datetime.today().weekday()
if (day_of_the_week == 1):
    print("buy dogecoin")
    action = "buy"
elif (day_of_the_week == 3):
    print("sell dogecoin - make sure you have dogecoin in your account to sell")
    action = "sell"
else:
    print("take no action")
    action = "none"

sell dogecoin - make sure you have dogecoin in your account to sell


# Check Dogecoin Historical Price Data

Note: this notebook assumes market orders are placed, but limit orders could be placed as well. 

In [204]:
#Dogecoin historical data collection

#https://alpaca.markets/learn/code-cryptocurrency-live-trading-bot-python-alpaca/
doge_data = api.get_crypto_bars("DOGEUSD", TimeFrame.Hour).df
doge_data.tail()
#exhanges are CBSE, ERSX, FTXU

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2022-09-22 15:00:00+00:00,FTXU,0.058697,0.059067,0.058697,0.058911,125008,14,0.058924
2022-09-22 16:00:00+00:00,FTXU,0.058787,0.059252,0.058653,0.059252,14759,12,0.059129
2022-09-22 17:00:00+00:00,FTXU,0.059253,0.059344,0.058974,0.059159,103629,18,0.059173
2022-09-22 18:00:00+00:00,FTXU,0.059089,0.059188,0.058988,0.059036,1146101,48,0.059113
2022-09-22 19:00:00+00:00,FTXU,0.059204,0.059432,0.059101,0.059101,325774,28,0.059205


# Check Current Dogecoin Position

Check to make sure the quantity of dogecoin you want to buy / sell matches up with your intentions. 

If you're selling Dogecoin, check to see if you have a fractional share of Dogecoin, rather than a whole number. 
During testing, it was noticed that buying 1 dogecoin actually resulted in the user buying a fractional coin. 

In [219]:
#Get dogecoin position
#If this statement errors out, your account currently doesn't have any dogecoin in it

#https://alpaca.markets/learn/code-cryptocurrency-live-trading-bot-python-alpaca/
# Get current position on Bitcoin. Yields error if we don't hold any
dogecoin_qty_to_sell = api.get_position('DOGEUSD').qty
print(f"your paper trading accout currently has {dogecoin_qty_to_sell} dogecoins")
if (float(dogecoin_qty_to_sell) < 1 ) & (float(dogecoin_qty_to_sell) > 0 ):
    print("\nyou have less than 1 dogecoin, you won't be able to sell this through the API because the size is too small. Increase the amount of dogecoin you have to > 1 in order to place a sell order without error.")

your paper trading accout currently has 1.99 dogecoins


In [220]:
# Place trades with Alpaca API 

if action == "buy":
    print("place buy order with alpaca api")
    api.submit_order('DOGEUSD', qty=1000, side='buy', time_in_force="gtc")
elif action == "sell":
    print("place sell order with alpaca api")
    api.submit_order('DOGEUSD', qty=dogecoin_qty_to_sell, side='sell', time_in_force="gtc")
else:
    print("take no action")

place sell order with alpaca api


# Check to Make Sure That Your Order Executed Successfully

You shouldn't have any open orders

Your closed orders should display information about your latest trade

In [223]:
# list all of your open orders: 

#https://forum.alpaca.markets/t/cancel-all-the-open-orders-using-python-api/2890/2

open_orders_list = api.list_orders(status='open')
open_orders_list

[]

In [222]:
# print closed orders

closed_orders = api.list_orders(
    status='closed',
    limit=100,
    nested=True  # show nested multi-leg orders
)

closed_orders = [o for o in closed_orders if o.symbol == 'TSLA']
print(closed_orders)

# Other Useful Functions

In [174]:
#Get account details

account = api.get_account()
account 

In [176]:
# print closed orders

closed_orders = api.list_orders(
    status='closed',
    limit=100,
    nested=True  # show nested multi-leg orders
)

closed_orders = [o for o in closed_orders if o.symbol == 'TSLA']
print(closed_orders)

In [ ]:
# list all of your open orders: 

#https://forum.alpaca.markets/t/cancel-all-the-open-orders-using-python-api/2890/2

open_orders_list = api.list_orders(status='open')
open_orders_list

In [ ]:
# cancel all of your open orders:

#https://forum.alpaca.markets/t/cancel-all-the-open-orders-using-python-api/2890/2

api.cancel_all_orders()

In [ ]:
# list how much you gained / lost today relative to yesterday: 

#https://alpaca.markets/deprecated/docs/api-documentation/how-to/account/
#Print your daily profit / loss

balance_change = float(account.equity) - float(account.last_equity)
balance_change